In [251]:
### Assumption : Confirmed_cases.csv exists in the working directory

from selenium import webdriver
import pandas as pd
import os
import re
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from datetime import datetime
import collections

In [252]:
URL = "https://www.arcgis.com/apps/opsdashboard/index.html#/85320e2ea5424dfaaa75ae62e5c06e61"

In [253]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
br = webdriver.Chrome(executable_path=r"C:/Users/qasim/Downloads/chromedriver_win32_v80/chromedriver.exe",options=chrome_options)
br.get(URL)
time.sleep(3)

In [254]:
data = br.find_element_by_id("ember21").text.split("\n")[:6]

In [255]:
def extract_num(string):
    string = string.split(" ")[0]
    return int(string.replace(",", ""))

In [256]:
logistics = {"CONFIRMED" : extract_num(data[1]), "DEATHS" : extract_num(data[3]), "RECOVERED" : extract_num(data[5])}

In [257]:
data = br.find_element_by_id("ember107").click()

In [258]:
TIME = str(datetime.now())
TIME = ':'.join(TIME.split(":")[:-1])

In [259]:
root = br.find_element_by_id("ember60")

In [260]:
br.find_element_by_id("ember66").screenshot("Images/"+ TIME.split(" ")[0] +".png")

True

In [261]:
data = root.text.split("\n")[1:]

In [262]:
case = {}

In [263]:
for x in data:
    x = x.split(" ")
    country = " ".join(x[1:]).replace("*", "").strip()
    cases = int(x[0].replace(",",""))
    case[country] = cases

In [264]:
case = collections.OrderedDict(sorted(case.items()))

In [265]:
raw_data = {**logistics, **case}
raw_data = collections.OrderedDict(sorted(raw_data.items()))

In [266]:
df = pd.read_csv("Confirmed_case.csv")

In [267]:
conts = list(df["Country"])
curr_count = len(df["Country"])

In [268]:
zero_fill = np.zeros(df.shape[1] - 1)

In [269]:
for label in raw_data:
    if(label not in conts):
        temp = np.append(zero_fill, label)
        temp = temp[::-1]
        df.loc[curr_count] = temp
df = df.sort_values(by=['Country'], inplace=False).reset_index(drop=True)

In [270]:
df.tail()

,Country,3/18/2020 18:53,3/18/2020 19:07,3/18/2020 19:17,3/18/2020 19:27,3/18/2020 20:09,3/18/2020 20:18,3/18/2020 20:28,3/18/2020 20:57,3/18/2020 21:07,...,2020-03-19 17:42,2020-03-19 17:52,2020-03-19 18:02,2020-03-19 18:12,2020-03-19 18:22,2020-03-19 18:32,2020-03-19 18:42,2020-03-19 18:52,2020-03-19 19:02,2020-03-19 19:12
98,United Kingdom,2642,2642,2642,2642,2642,2642,2642,2642,2642,...,2716,2716,2716,2716,2716,2716,2716,2716,2716,2716
99,Uruguay,50,50,50,50,50,50,50,50,50,...,79,79,79,79,79,79,79,79,79,79
100,Uzbekistan,0,0,0,0,0,0,0,0,0,...,85,85,85,85,85,85,85,85,85,85
101,Venezuela,36,36,36,36,36,36,36,36,36,...,23,23,23,23,23,23,23,23,23,23
102,Vietnam,75,75,75,75,75,75,75,75,75,...,42,42,42,42,42,42,42,42,42,42


In [271]:
df_temp = pd.DataFrame.from_dict(raw_data, orient="index", columns=[TIME]).reset_index()

In [272]:
del df_temp["index"]

In [273]:
df_cum = [df, df_temp]

In [274]:
df_cum = pd.concat(df_cum, axis=1)

In [275]:
df_cum.tail()

,Country,3/18/2020 18:53,3/18/2020 19:07,3/18/2020 19:17,3/18/2020 19:27,3/18/2020 20:09,3/18/2020 20:18,3/18/2020 20:28,3/18/2020 20:57,3/18/2020 21:07,...,2020-03-19 17:52,2020-03-19 18:02,2020-03-19 18:12,2020-03-19 18:22,2020-03-19 18:32,2020-03-19 18:42,2020-03-19 18:52,2020-03-19 19:02,2020-03-19 19:12,2020-03-19 19:16
98,United Kingdom,2642,2642,2642,2642,2642,2642,2642,2642,2642,...,2716,2716,2716,2716,2716,2716,2716,2716,2716,2716
99,Uruguay,50,50,50,50,50,50,50,50,50,...,79,79,79,79,79,79,79,79,79,79
100,Uzbekistan,0,0,0,0,0,0,0,0,0,...,85,85,85,85,85,85,85,85,85,23
101,Venezuela,36,36,36,36,36,36,36,36,36,...,23,23,23,23,23,23,23,23,23,42
102,Vietnam,75,75,75,75,75,75,75,75,75,...,42,42,42,42,42,42,42,42,42,85


In [277]:
null_values = df_cum[df_cum[TIME].isnull()].index.tolist()

In [278]:
if(len(null_values) > 0):
    priori = df_cum.columns[-2]
    df_copy = df_cum.copy()

    for x in null_values:
        value = df_copy[priori][x]
        df_cum.loc[:,TIME][x] = value

    del df_copy

In [280]:
df_cum.tail(9)

,Country,3/18/2020 18:53,3/18/2020 19:07,3/18/2020 19:17,3/18/2020 19:27,3/18/2020 20:09,3/18/2020 20:18,3/18/2020 20:28,3/18/2020 20:57,3/18/2020 21:07,...,2020-03-19 17:52,2020-03-19 18:02,2020-03-19 18:12,2020-03-19 18:22,2020-03-19 18:32,2020-03-19 18:42,2020-03-19 18:52,2020-03-19 19:02,2020-03-19 19:12,2020-03-19 19:16
94,Tunisia,29,29,29,29,29,29,29,29,29,...,39,39,39,39,39,39,39,39,39,39
95,Turkey,98,98,98,98,98,98,98,98,98,...,192,192,192,192,192,192,192,192,192,192
96,US,7769,7769,7769,7769,7769,7769,7769,7769,7769,...,13159,13159,13159,13159,13159,13159,13159,13159,13159,13159
97,United Arab Emirates,113,113,113,113,113,113,113,113,113,...,140,140,140,140,140,140,140,140,140,140
98,United Kingdom,2642,2642,2642,2642,2642,2642,2642,2642,2642,...,2716,2716,2716,2716,2716,2716,2716,2716,2716,2716
99,Uruguay,50,50,50,50,50,50,50,50,50,...,79,79,79,79,79,79,79,79,79,79
100,Uzbekistan,0,0,0,0,0,0,0,0,0,...,85,85,85,85,85,85,85,85,85,23
101,Venezuela,36,36,36,36,36,36,36,36,36,...,23,23,23,23,23,23,23,23,23,42
102,Vietnam,75,75,75,75,75,75,75,75,75,...,42,42,42,42,42,42,42,42,42,85


In [40]:
df_cum.to_csv("Confirmed_case.csv", index=False)